In [8]:
import matplotlib.pyplot as plt
import numpy as np
import warnings
import tarfile
import cv2
import os

from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize

warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
base_path = "/dataset/dataset/"

In [3]:
train_files = []
with open(base_path + "train.txt") as fp:
    for line in fp:
        train_files.append(line.strip())

In [4]:
# for file_name in train_files:
#     tar = tarfile.open(base_path + "data/" + file_name + '.tar')
#     tar.extractall(path=base_path + "train/")

In [35]:
X_train = np.zeros((21100, 256, 256, 1), dtype=np.uint8)
y_train = np.zeros((21100, 256, 256, 1), dtype=np.bool)

for n, file_name in enumerate(train_files):
    mask = imread(base_path + "masks/" + file_name + ".png", 0)
    mask = np.expand_dims(resize(mask, (256, 256), mode='constant', preserve_range=True), axis=-1)
    files = os.listdir(base_path + "train/data/" + file_name + '/')
    for i in range(len(files)):
        y_train[n] = mask
    for file in files:
        img = imread(base_path + "train/data/" + file_name + '/' + file) # 0 for gray scale
        img = np.expand_dims(resize(img, (256, 256), mode="constant", preserve_range=True), axis=-1)
        X_train[n] = img

In [36]:
assert X_train.shape == y_train.shape
assert len(X_train) == len(y_train)

In [26]:
# plt.figure(figsize=(25, 25))
# plt.subplot(131)
# plt.imshow(X_train[0])
# plt.subplot(132)
# plt.imshow(y_train[0])

In [37]:
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

From https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277

In [38]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [43]:
inputs = Input((256, 256, 1))
s = Lambda(lambda x: x / 255)(inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(s)
c1 = Dropout(0.1)(c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c1)
p1 = MaxPooling2D((2, 2))(c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
# model.summary()

In [40]:
from keras.utils import plot_model

In [41]:
plot_model(model, to_file='images/model_unet.png', show_shapes=True)

In [42]:
earlystopper = EarlyStopping(patience=5, verbose=1)
# checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
results = model.fit(X_train, y_train, validation_split=0.2, batch_size=128, epochs=10, callbacks=[earlystopper])

Train on 16880 samples, validate on 4220 samples
Epoch 1/10
16880/16880 [==============================] - 3928s 233ms/step - loss: 0.0289 - mean_iou: 0.5396 - val_loss: 4.9113e-07 - val_mean_iou: 0.5373
Epoch 2/10
16880/16880 [==============================] - 3911s 232ms/step - loss: 0.0088 - mean_iou: 0.5325 - val_loss: 2.9728e-07 - val_mean_iou: 0.5298
Epoch 3/10
16880/16880 [==============================] - 3915s 232ms/step - loss: 0.0087 - mean_iou: 0.5283 - val_loss: 1.6479e-07 - val_mean_iou: 0.5261
Epoch 4/10
16880/16880 [==============================] - 3914s 232ms/step - loss: 0.0087 - mean_iou: 0.5245 - val_loss: 1.3104e-07 - val_mean_iou: 0.5232
Epoch 5/10
16880/16880 [==============================] - 3914s 232ms/step - loss: 0.0087 - mean_iou: 0.5219 - val_loss: 1.2164e-07 - val_mean_iou: 0.5209
Epoch 6/10
16880/16880 [==============================] - 3915s 232ms/step - loss: 0.0087 - mean_iou: 0.5203 - val_loss: 1.1895e-07 - val_mean_iou: 0.5198
Epoch 7/10
16880/1688